In [1]:
# !pip install netCDF4 pyarrow

In [2]:
from glob import glob
from hashlib import md5

import pandas as pd
import xarray as xr

In [3]:
paths = glob('../../data/*.nc')

In [4]:
dfs = []
for path in paths:
    dset = xr.load_dataset(path)
    df = dset[['active_trials', 'contrast_left', 'contrast_right', 'stim_onset', 'gocue', 'response_type', 'response_time', 'feedback_time', 'feedback_type', 'reaction_time', 'reaction_type']].to_dataframe().reset_index()
    # df = df[df['active_trials']].drop(columns=['active_trials'])
    df = df.rename(columns={'gocue': 'gocue_time'})
    df = df.assign(
        mouse=dset.attrs['mouse'], 
        session_date=dset.attrs['session_date'], 
        session_id=str(md5((dset.attrs['mouse'] + dset.attrs['session_date']).encode()).hexdigest())[:6],
    )
    dfs.append(df)
df = pd.concat(dfs, ignore_index=True)
df.head()


,trial,active_trials,contrast_left,contrast_right,stim_onset,gocue_time,response_type,response_time,feedback_time,feedback_type,reaction_time,reaction_type,mouse,session_date,session_id
0,1,True,100,0,0.5,1.027216,1.0,1.150204,1.186819,1.0,170.0,1.0,Cori,2016-12-14,5dd41e
1,2,True,0,50,0.5,0.874414,-1.0,1.399503,1.437623,1.0,230.0,-1.0,Cori,2016-12-14,5dd41e
2,3,True,100,50,0.5,0.825213,1.0,0.949291,0.986016,1.0,200.0,1.0,Cori,2016-12-14,5dd41e
3,4,True,0,0,0.5,0.761612,0.0,2.266802,2.296436,1.0,860.0,1.0,Cori,2016-12-14,5dd41e
4,5,True,50,100,0.5,0.662010,1.0,0.816776,0.827613,-1.0,140.0,1.0,Cori,2016-12-14,5dd41e


In [5]:
df_winter2016 = df[df.session_date.isin(
    ['2016-12-14', '2016-12-17', '2016-12-18', 
     '2017-01-07', '2017-01-08', '2017-01-09', '2017-01-10', '2017-01-11', '2017-01-12',
    ]
)]
df_winter2016.to_csv('steinmetz_winter2016.csv', index=False)

In [6]:
df_summer2017 = df[df.session_date.isin(
    ['2017-05-15', '2017-05-16', '2017-05-18',
    '2017-06-15', '2017-06-16', '2017-06-17', '2017-06-18',]
)]
df_summer2017.to_csv('steinmetz_summer2017.csv', index=False)
# df_summer2017.head()

In [7]:
df_winter2017 = df[df.session_date.isin(
    ['2017-10-11', '2017-10-29', '2017-10-30', '2017-10-31',
     '2017-11-01', '2017-11-02', '2017-11-04', '2017-11-05',
     '2017-12-05', '2017-12-06', '2017-12-07', '2017-12-08',   '2017-12-09', '2017-12-10', '2017-12-11'
    ]
)]
df_winter2017.to_csv('steinmetz_winter2017.csv', index=False)

In [8]:
df.to_parquet('steinmetz_day1.parquet')
df.to_csv('steinmetz_day1.csv')
df.to_csv('steinmetz_day1_compressed.csv', compression="gzip")

In [9]:
df.session_date.unique()

array(['2016-12-14', '2016-12-17', '2016-12-18', '2017-01-07',
       '2017-01-08', '2017-01-09', '2017-01-10', '2017-01-11',
       '2017-01-12', '2017-05-15', '2017-05-16', '2017-05-18',
       '2017-06-15', '2017-06-16', '2017-06-17', '2017-06-18',
       '2017-10-11', '2017-10-29', '2017-10-30', '2017-10-31',
       '2017-11-01', '2017-11-02', '2017-11-04', '2017-11-05',
       '2017-12-05', '2017-12-06', '2017-12-07', '2017-12-08',
       '2017-12-09', '2017-12-10', '2017-12-11'], dtype=object)